In [1]:
import pandas as pd
from torch.utils.data import DataLoader
#from pytorch_lightning.profiler import Profiler, AdvancedProfiler
import HeatfluxData as hfd
import HeatfluxModel as hfm

/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filename_data = f'../students/bogdaale/NN-examples/scaled_QdataKn22width100microns.csv'
# Using heat flux data module
scaled_Qdata = pd.read_csv(filename_data, header=0, index_col=0)
test_set, train_set, validation_set, vis_set = hfd.heat_flux_datasets(scaled_Qdata, 0.3, 0.8)